In [ ]:
#setting connection to google drive 
from google.colab import  drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#changing the direcctory 
%cd "/content/gdrive/MyDrive/fake news"

/content/gdrive/MyDrive/fake news


In [ ]:
#libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.layers import Dense,LSTM,Dropout,Embedding,Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
import re
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import nltk
nltk.download('stopwords')
from sklearn.metrics import  confusion_matrix,accuracy_score




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#view of data
vocab_size=5000
df=pd.read_csv("train.csv")
df["content"]=df['author']+' '+df['title']
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
#droppinf the na values
df=df.dropna()
df.reset_index(inplace=True)

In [ ]:
# stemming and cleaning the data 
ps=PorterStemmer()
corpus=[]
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['content'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)




In [ ]:
#one hot vector 
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 

In [ ]:
#padding the sequence
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
y=df["label"]
X_final=np.array(embedded_docs)
y_final=np.array(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=0)



In [ ]:
#defining the model lstm 
model=Sequential()
model.add(Embedding(vocab_size,40,input_length=30))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam", loss = BinaryCrossentropy(from_logits=True),metrics ="accuracy" )

In [ ]:
y=df["label"]
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
#fitiing the model
calback=[EarlyStopping(monitor="val_loss",min_delta=0.001,patience=5,verbose=1),
         ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=2,verbose=1),
         ModelCheckpoint("classify.h5",monitor="val_loss",save_best_only=True)]
history = model.fit(X_train, y_train, epochs = 25,batch_size = 128,validation_data = (X_test,y_test),callbacks = calback)

Epoch 1/25


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


96/96 [==============================] - 10s 80ms/step - loss: 0.3148 - accuracy: 0.8548 - val_loss: 0.0754 - val_accuracy: 0.9745
Epoch 2/25
96/96 [==============================] - 7s 77ms/step - loss: 0.0511 - accuracy: 0.9847 - val_loss: 0.0576 - val_accuracy: 0.9829
Epoch 3/25
96/96 [==============================] - 7s 76ms/step - loss: 0.0170 - accuracy: 0.9954 - val_loss: 0.0663 - val_accuracy: 0.9798
Epoch 4/25
96/96 [==============================] - 7s 76ms/step - loss: 0.0087 - accuracy: 0.9973 - val_loss: 0.0672 - val_accuracy: 0.9765

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/25
96/96 [==============================] - 7s 76ms/step - loss: 0.0050 - accuracy: 0.9989 - val_loss: 0.0635 - val_accuracy: 0.9833
Epoch 6/25
96/96 [==============================] - 7s 76ms/step - loss: 0.0023 - accuracy: 0.9995 - val_loss: 0.0635 - val_accuracy: 0.9836

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Ep

In [ ]:
# evaluating the test set 
 model.evaluate(X_test,y_test)

189/189 [==============================] - 1s 8ms/step - loss: 0.0751 - accuracy: 0.9829


[0.07510475814342499, 0.9829328656196594]

In [ ]:
p# predicting the values
y_pred=model.predict_on_batch(X_test)

In [ ]:
# confusion matrix
from tensorflow.math import round,confusion_matrix
y_pred=round(y_pred)
confusion_matrix(y_test,y_pred)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3366,   23],
       [  80, 2566]], dtype=int32)>

In [ ]:
 from tensorflow.keras.layers import Bidirectional

In [ ]:
#bidirectional lastm
model=Sequential()
model.add(Embedding(vocab_size,40,input_length=30))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam", loss = BinaryCrossentropy(from_logits=True),metrics ="accuracy" )

In [ ]:
calback=[EarlyStopping(monitor="val_loss",min_delta=0.001,patience=5,verbose=1),
         ReduceLROnPlateau(monitor="val_loss",factor=0.5,patience=2,verbose=1),
         ModelCheckpoint("classify.h5",monitor="val_loss",save_best_only=True)]
history = model.fit(X_train, y_train, epochs = 25,batch_size = 128,validation_data = (X_test,y_test),callbacks = calback)

Epoch 1/25


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


96/96 [==============================] - 17s 139ms/step - loss: 0.3137 - accuracy: 0.8437 - val_loss: 0.1004 - val_accuracy: 0.9635
Epoch 2/25
96/96 [==============================] - 12s 128ms/step - loss: 0.0461 - accuracy: 0.9863 - val_loss: 0.0594 - val_accuracy: 0.9793
Epoch 3/25
96/96 [==============================] - 12s 127ms/step - loss: 0.0172 - accuracy: 0.9949 - val_loss: 0.0723 - val_accuracy: 0.9776
Epoch 4/25
96/96 [==============================] - 12s 128ms/step - loss: 0.0111 - accuracy: 0.9972 - val_loss: 0.0581 - val_accuracy: 0.9853
Epoch 5/25
96/96 [==============================] - 12s 129ms/step - loss: 0.0049 - accuracy: 0.9985 - val_loss: 0.0636 - val_accuracy: 0.9836
Epoch 6/25
96/96 [==============================] - 12s 127ms/step - loss: 0.0039 - accuracy: 0.9989 - val_loss: 0.0805 - val_accuracy: 0.9786

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/25
96/96 [==============================] - 12s 126ms/step - los

In [ ]:
y_pred=model.predict_on_batch(X_test)
y_pred=round(y_pred)
confusion_matrix(y_test,y_pred)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3364,   25],
       [  58, 2588]], dtype=int32)>